In [2]:
# !pip install lusid-drive-sdk

  Using cached lusid_drive_sdk-0.1.410-py3-none-any.whl (74 kB)


In [119]:
# a randomly selected subset of 25 instruments
# subset_file = open('_data/ftse_subset.csv').read()

In [129]:
# # upload subset file to drive so we can query l8r
# import lusid_drive
# from lusid_drive.rest import ApiException
# from pprint import pprint

# drive_config = lusid_drive.Configuration(
#     host = "https://fbn-uni.lusid.com/drive"
# )
# drive_config.access_token = RefreshingToken()

# # Enter a context with an instance of the API client
# with lusid_drive.ApiClient(drive_config) as api_client:
#     # Create an instance of the API class
#     api_instance = lusid_drive.FoldersApi(api_client)
#     create_folder = {"path":"/","name":"finbourne university"} # CreateFolder | A CreateFolder object that defines the name and path of the new folder

#     try:
#         # [EARLY ACCESS] CreateFolder: Create a new folder in LUSID Drive
#         api_response = api_instance.create_folder(create_folder)
#         pprint(api_response)
#     except ApiException as e:
#         print("Exception when calling FoldersApi->create_folder: %s\n" % e)
        
#     api_instance = lusid_drive.FilesApi(api_client)
#     x_lusid_drive_filename = 'ftse_subset.csv' # str | File name.
#     x_lusid_drive_path = '/finbourne university/' # str | File path.
#     content_length = len(subset_file.encode('UTF-8'))
#     body = subset_file # str | 

#     try:
#         # [EARLY ACCESS] CreateFile: Uploads a file to Lusid Drive. If using an SDK, consider using the UploadAsStreamAsync function for larger files instead.
#         api_response = api_instance.create_file(x_lusid_drive_filename, x_lusid_drive_path, content_length, body)
#         pprint(api_response)
#     except ApiException as e:
#         print("Exception when calling FilesApi->create_file: %s\n" % e)
        


Exception when calling FoldersApi->create_folder: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 02 Nov 2022 15:15:53 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'drive-meta-success': 'False', 'drive-meta-requestId': '0HMLROQKIKPU9:0000006B', 'lusid-meta-correlationId': '0HMLROQKIKPU9:0000006B', 'drive-meta-duration': '1387', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; font-src data: 'self' https://fonts.googleapis.com https://fonts.gstatic.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com; style-src https://fonts.googleapis.com 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com; connect-src https://finbourne.okta.com 'self' https://*.lusid

In [2]:
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
import lusid as lu
import pandas as pd


secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = ApiClientFactory(
    token = RefreshingToken(), 
    api_secrets_filename = secrets_path,
    app_name = "LusidJupyterNotebook"
)

# Confirm success by printing SDK version
api_status = pd.DataFrame(api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict())
display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.10220.0,0.5.3064,"{'relation': 'RequestLogs', 'href': 'http://fb..."


In [3]:
ftse_instruments = pd.read_csv('_data/ftse100.csv')
nasdaq_instruments = pd.read_csv('_data/nasdaq100.csv')

In [6]:
from lusid.rest import ApiException
from pprint import pprint
# create portfolio for this module
def upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description):
    api_instance = api_factory.build(lu.TransactionPortfoliosApi)
        # Create an instance of the API class
    create_transaction_portfolio_request = {"displayName":portfolio_name,"description":portfolio_description,"code":code,"baseCurrency":"GBP", "instrumentScopes":["FBNUniversityModule-T2-2"]} # CreateTransactionPortfolioRequest | The definition of the transaction portfolio.
    try:
        # UpdatePortfolio: Update portfolio
        api_response = api_instance.create_portfolio(scope, create_transaction_portfolio_request)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling PortfoliosApi->update_portfolio: %s\n" % e)

scope = 'FBNUniversity' # str | The scope of the portfolio.
code = 'Module-T2-2' # str | The code of the portfolio. Together with the scope this uniquely identifies the portfolio.
portfolio_name = "Training module T2.2"
portfolio_description = "A portfolio to hold transactions"
upsert_portfolio(api_factory, scope, code, portfolio_name, portfolio_description)

Exception when calling PortfoliosApi->update_portfolio: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Thu, 03 Nov 2022 20:33:16 GMT', 'Content-Type': 'application/problem+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'X-Rate-Limit-Limit': '1m', 'X-Rate-Limit-Remaining': '4998', 'X-Rate-Limit-Reset': '2022-11-03T20:34:06.5231677Z', 'lusid-meta-success': 'False', 'lusid-meta-requestId': '0HMLTOJP7QOIS:00000022', 'lusid-meta-correlationId': '0HMLTOJP7QOIS:00000022', 'lusid-meta-duration': '154', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'Server': 'FINBOURNE', 'Content-Security-Policy': "default-src 'self' https://*.lusid.com https://*.finbourne.com; script-src 'unsafe-inline' 'self' https://*.lusid.com https://*.finbourne.com https://editor.swagger.io; font-src 'self' fonts.googleapis.com; img-src data: 'self' https://*.lusid.com https://*.finbourne.com https://validator.swagger.io; style-src 'unsafe-inline' 'self'

In [7]:
# create a request to upsert a bunch of instruments.
def generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier):
    request_id = name + isin
    request ={"name":name,
                     "identifiers":{ "Figi":{"value":figi,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"},
                                     "Isin":{"value":isin,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"},
                                     other_identifier_name:{"value":other_identifier,"effectiveAt":"0001-01-01T00:00:00.0000000+00:00"}
                                   },

                     "definition":{"instrumentType":"Equity", "domCcy":ccy}}
    return request_id, request

In [8]:
# Create an instance of the API class
api_instance = api_factory.build(lu.InstrumentsApi)
other_identifier_name = 'Sedol'
ccy = 'GBP'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in ftse_instruments.itertuples())
ftse_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}
other_identifier_name = 'Cusip'
ccy = 'USD'
upsert_body_tuple = (generate_upsert_instrument_request(api_instance, name, ccy, figi, isin, other_identifier_name, other_identifier)
               for 
               index, ticker, name, _, isin, other_identifier, figi
               in nasdaq_instruments.itertuples())
nasdaq_body_dict = {request_id:request_body for request_id,request_body in upsert_body_tuple}

request_body_dict = nasdaq_body_dict | ftse_body_dict

try:
    # UpsertInstruments: Upsert instruments
    api_response = api_instance.upsert_instruments(request_body_dict, scope=scope)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling InstrumentsApi->upsert_instruments: %s\n" % e)


{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-uni.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://fbn-uni.lusid.com/app/insights/logs/0HMLTOJOK0URF:0000001C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Created Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['ExperianGB00B19NLV48',
                 'BT GroupGB0030913577',
                 'Spirax-Sarco EngineeringGB00BWFGQN14',
                 'B&MLU1072616219',
                 'EntainIM00B5VQMV65',
                 'Persimmon plcGB0006825383',
                 'GlaxoSmithKlineGB0009252882',
                 'RSA Insurance Group

# Luminesce
## What is Structured Query Language (SQL)
SQL - pronounced "sequel" was first developed at IBM in the early 1970s, as a way to query and manipulate structured relational databases. SQL is a set-based, declarative programming language, however many implementations of SQL add extensions with procedural programming functionality, such as control-of-flow contructs.
## Luminesce Queries - SQL with extensions
SQLite is the most widely deployed database engine, used by several of the top web browsers, operating systems and application software.

Luminesce supports most of the query syntax for the SQLite dialect of SQL, but not the data manipulation or data definition syntax. Data can still be manipulated using some extensions to the language that Luminesce adds. 

Luminesce adds extensions to the SQLite implementation which can:
- exclude columns from queries
- automatically select important columns in queries
- parameterise queries
- write data to some Luminesce providers
- wait on results of some dependent operation, or for a time period
- run queries in parallel on an iterated range of parameters.
## Using Luminesce to query and join data from disparate sources, and to distribute the resulting information.
I'd like to read a file from Drive and use the data from this file to filter transaction results. I'd like to join these filtered results with another provider and distribute the results. This will be packaged up into a simple to use View before Scheduling the view.

### A query using a simple SELECT statement
Its simple to get data from a provider using a select statement.

Use the following statement to get all data from a provider:

`SELECT * FROM SOME.PROVIDER`

Or retrieve data from a subset of columns by specifying them:

`SELECT A_COLUMN, ANOTHER_COLUMN FROM SOME.PROVIDER`

Here we'll use a SELECT statement to query for all of our porfolios.

In [11]:
%%luminesce
SELECT * FROM Lusid.Portfolio

,PortfolioScope,PortfolioCode,PortfolioType,BaseCurrency,DisplayName,Description,OriginPortfolioScope,OriginPortfolioCode,ParentPortfolioScope,ParentPortfolioCode,ChildPortfolioIds,CorporateActionSourceScope,CorporateActionSourceId,AccountingMethod,SubHoldingKeys,Created,InstrumentScopes
0,FBNUniversity,Module-6-2,Transaction,GBP,"Training module 6.2 - Same transaction code, d...",NaN,FBNUniversity,Module-6-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-2
1,FBNUniversity,Module-4-1,Transaction,GBP,Training module 4.1,NaN,FBNUniversity,Module-4-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-4-1
2,FBNUniversity,Module-3-1,Transaction,GBP,Training module 3.1,NaN,FBNUniversity,Module-3-1,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/Module-3-1,2022-01-01,FBNUniversityModule-3-1
3,FBNUniversity,Module-T2-2,Transaction,GBP,Training module T2.2,A portfolio to hold transactions,FBNUniversity,Module-T2-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-11-03 20:14:15.334,FBNUniversityModule-T2-2
4,FBNUniversity,Module-3-2,Transaction,GBP,Training module 3.2,NaN,FBNUniversity,Module-3-2,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-3-2
5,FBNUniversity,Module-1,Transaction,GBP,Module 1,NaN,FBNUniversity,Module-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2021-12-01,FBNUniversityModule-1
6,FBNUniversity,Module-6-3,Transaction,GBP,Training module 6.3 - Reduce cash balance by c...,NaN,FBNUniversity,Module-6-3,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-3
7,FBNUniversity,Module-6-4,Transaction,GBP,Training module 6.4 - Track commission as sepa...,NaN,FBNUniversity,Module-6-4,NaN,NaN,NaN,NaN,NaN,Default,Transaction/FBNUniversity/TrackCommission,2022-01-01,FBNUniversityModule-6-4
8,FBNUniversity,Module-6-1,Transaction,GBP,Training module 6.1 - Different transaction co...,NaN,FBNUniversity,Module-6-1,NaN,NaN,NaN,NaN,NaN,Default,NaN,2022-01-01,FBNUniversityModule-6-1


We can specify column names if we are only interested in some parts of the data - here I really just want to see the name of the portfolio and the portfolio code:

In [12]:
%%luminesce
SELECT DisplayName, PortfolioCode FROM Lusid.Portfolio

,DisplayName,PortfolioCode
0,Training module 3.2,Module-3-2
1,Training module 4.1,Module-4-1
2,Training module T2.2,Module-T2-2
3,Module 1,Module-1
4,Training module 3.1,Module-3-1
5,"Training module 6.2 - Same transaction code, d...",Module-6-2
6,Training module 6.1 - Different transaction co...,Module-6-1
7,Training module 6.3 - Reduce cash balance by c...,Module-6-3
8,Training module 6.4 - Track commission as sepa...,Module-6-4


### Limiting the number of return results using LIMIT
These simple SELECT statements are useful for getting an idea of the shape and quality of the data returned by a provider, but they tend to return a lot of data. Lets use LIMIT to reduce the amount of data requested:

In [13]:
%%luminesce
SELECT DisplayName, PortfolioCode FROM Lusid.Portfolio LIMIT 5

,DisplayName,PortfolioCode
0,"Training module 6.2 - Same transaction code, d...",Module-6-2
1,Training module 4.1,Module-4-1
2,Training module 3.1,Module-3-1
3,Training module T2.2,Module-T2-2
4,Training module 3.2,Module-3-2


### filter queries WHERE ...
We can now filter these results down - lets get all of the Equities that are listed in GBP


In [96]:
%%luminesce
SELECT * FROM Lusid.Instrument.Equity WHERE DomCcy = 'GBP'

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D6G,GB00BH0P3Z91,BH0P3Z9,NaN,NaN,NaN,BBG000F3W0C7,NaN,NaN,NaN,...,default,BHP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
1,LUID_00003D5B,GB0031743007,NaN,NaN,BRBY LN,imd_43599356,BBG000NSXQ99,NaN,NaN,NaN,...,FBNUniversityModule-1,Burberry,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
2,LUID_00003DBV,GB00B18V8630,B18V863,NaN,NaN,NaN,BBG000H90Q58,NaN,NaN,NaN,...,fbn-uni-T02002,Pennon Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
3,LUID_00003D6K,GB0030913577,3091357,NaN,NaN,NaN,BBG000C05R82,NaN,NaN,NaN,...,default,BT Group,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D9C,GB0000811801,81180,NaN,NaN,NaN,BBG000BD6DG6,NaN,NaN,NaN,...,fbn-uni-T02002,Barratt Developments,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,LUID_00003D55,JE00B4T3BW64,NaN,NaN,GLEN LN,imd_34534555,BBG001MM1KV4,NaN,NaN,NaN,...,FBNUniversityModule-1,Glencore,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
162,LUID_00003D5O,NaN,NaN,NaN,NaN,NaN,BBG000C0M8X7,NaN,NaN,NaN,...,FBNUniversityModule-3-2,Unilever,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
163,LUID_00003D4W,GB0004052071,NaN,NaN,HLMA LN,imd_78009283,BBG000BDLT47,NaN,NaN,NaN,...,FBNUniversityModule-1,Halma,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
164,LUID_00003D9L,GB0007980591,798059,NaN,NaN,NaN,BBG000C05BD1,NaN,NaN,NaN,...,fbn-uni-T02002,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP


### JOINing disparate datasources with join
Now, I've got a file in drive with sector information for some of these instruments. I'm gonna join data from the file using our Drive provider with the saved instrument data so I can see the sectors for instruments where we know this information.

First lets get our instrument data.

Then we'll grab the file contents from drive.

We'll take a look at cross-joins, which Sqllite uses to build left joins and inner joins. Cross-joins return the cartesian product of both tables - each row on the left table is concatenated with every row on the right table, to form a massive table with every combination of both tables' rows:


In [10]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities CROSS JOIN @x as File

,DisplayName,DomCcy,Sector
0,eBay Inc.,USD,Financial Services
1,eBay Inc.,USD,General Industrials
2,eBay Inc.,USD,Travel & Leisure
3,eBay Inc.,USD,Equity Investment Instruments
4,eBay Inc.,USD,Household Goods & Home Construction
...,...,...,...
11795,Experian,GBP,Mining
11796,Experian,GBP,Media
11797,Experian,GBP,Mobile Telecommunications
11798,Experian,GBP,Food & Drug Retailers


In a left join, all rows from the left table are returned, along with rows in the right table that match the supplied key constraint. If there is a row in the left table with no matching row on the right, the columns that come from the right table in the resulting row are set to null.

We see this here as there are some companies with no matching sector in our file - these are returned as null.

In [11]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities LEFT JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector
0,Adobe Inc.,USD,NaN
1,DocuSign,USD,NaN
2,Mondi,GBP,NaN
3,Diageo,GBP,NaN
4,Starbucks Corporation,USD,NaN
...,...,...,...
467,Standard Life Aberdeen,GBP,NaN
468,Intertek,GBP,NaN
469,Polymetal International,GBP,NaN
470,Land Securities,GBP,Real Estate Investment Trusts


We are going to ignore instruments without a matching sector for now, so we'll use an inner join.
Inner joins only return rows where the joining key has a match on both sides of the table:

In [12]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT DisplayName, DomCcy, File.Sector FROM Lusid.Instrument.Equity as Equities INNER JOIN @x as File on Equities.Figi = File.Figi

,DisplayName,DomCcy,Sector
0,Sage Group,GBP,Software & Computer Services
1,Land Securities,GBP,Real Estate Investment Trusts
2,Intermediate Capital Group,GBP,Investment Services
3,DS Smith,GBP,General Industrials
4,British Land,GBP,Real Estate Investment Trusts
...,...,...,...
65,Legal & General,GBP,Life Insurance
66,British Land,GBP,Real Estate Investment Trusts
67,Aviva,GBP,Life Insurance
68,Glencore,GBP,Mining


### GROUPing results BY something in query
We can also group our results by a column - we'll perform an aggregation and group-by to count the number of instrument in each sector.

In [137]:
%%luminesce

/*
    We'll cover using variables later in the tutorial
*/

@x = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

SELECT count(File.Sector), File.Sector
FROM Lusid.Instrument.Equity as Equities 
Join @x as File on Equities.Figi = File.Figi
GROUP BY File.Sector

,count(File_Sector COLLATE BINARY),Sector
0,1,Automobiles & Parts
1,1,Equity Investment Instruments
2,2,Financial Services
3,1,Food & Drug Retailers
4,1,General Industrials
5,2,Household Goods & Home Construction
6,1,Investment Services
7,5,Life Insurance
8,1,Media
9,5,Mining


## Introducing some Luminesce extensions
### Using ^ to SELECT the most important columns from a provider
Let's use the carat symbol to grab then more important columns in the Equity provider. The list of important columns is decided by the writer of the Provider.

In [138]:
%%luminesce
SELECT ^ FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive
0,LUID_00003D6G,default,BHP,Equity,Equities,GBP,True
1,LUID_00003D5B,FBNUniversityModule-1,Burberry,Equity,Equities,GBP,True
2,LUID_00003DBV,fbn-uni-T02002,Pennon Group,Equity,Equities,GBP,True
3,LUID_00003D6K,default,BT Group,Equity,Equities,GBP,True
4,LUID_00003D72,fbn-uni-T02002,Broadcom Inc.,Equity,Equities,USD,True
...,...,...,...,...,...,...,...
265,LUID_00003D5O,FBNUniversityModule-3-2,Unilever,Equity,Equities,GBP,True
266,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True
267,LUID_00003D92,fbn-uni-T02002,Amazon.com,Equity,Equities,USD,True
268,LUID_00003D9L,fbn-uni-T02002,BP,Equity,Equities,GBP,True


### Excluding columns from the results of a query using the except keyword.
We can also exclude columns from the result of a query by using the except keyword:

In [143]:
%%luminesce
SELECT ^ (except Type, IsActive) FROM Lusid.Instrument.Equity

,LusidInstrumentId,Scope,DisplayName,InferredAssetClass,InferredDomCcy
0,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equities,GBP
1,LUID_00003DB6,fbn-uni-T02002,CSX Corporation,Equities,USD
2,LUID_00003DAL,fbn-uni-T02002,American Electric Power,Equities,USD
3,LUID_00003D5O,FBNUniversityModule-3-2,Unilever,Equities,GBP
4,LUID_00003D6X,default,Flutter Entertainment,Equities,GBP
...,...,...,...,...,...
265,LUID_00003DCG,fbn-uni-T02002,Admiral Group,Equities,GBP
266,LUID_00003D8C,fbn-uni-T02002,Aveva,Equities,GBP
267,LUID_00003D7F,fbn-uni-T02002,Cerner Corporation,Equities,USD
268,LUID_00003D9F,fbn-uni-T02002,Vertex Pharmaceuticals,Equities,USD


### Using parameters to filter queries
Many providers will have parameters that can also be used to filter a Luminesce query as part of a standard WHERE clause.

We can see these parameters by querying the sys.Field Provider.

In [146]:
%%luminesce
select FieldName, DataType, ParamDefaultValue, Description from Sys.Field where TableName = 'Lusid.Instrument.Equity' and FieldType = 'Parameter';

,FieldName,DataType,ParamDefaultValue,Description
0,AsAt,DateTime,0001-01-01T00:00:00Z,As at time to query at. Defaults to latest.
1,EffectiveAt,DateTime,0001-01-01T00:00:00Z,Effective time to query at. Defaults to latest.
2,UseLusidFilter,Boolean,True,Should the filter be translated to a Finbourne...


In [151]:
%%luminesce

select * from Lusid.Instrument.Equity where UseLusidFilter = False

,LusidInstrumentId,Isin,Sedol,Cusip,Ticker,ClientInternal,Figi,CompositeFigi,ShareClassFigi,Wertpapier,...,Scope,DisplayName,LookThroughPortfolioScope,LookThroughPortfolioCode,Type,InferredAssetClass,InferredDomCcy,IsActive,Identifiers,DomCcy
0,LUID_00003D76,US2855121099,NaN,285512109,NaN,NaN,BBG000BP0KQ8,NaN,NaN,NaN,...,fbn-uni-T02002,Electronic Arts,NaN,NaN,Equity,Equities,USD,True,NaN,USD
1,LUID_00003D7V,NL0009538784,NaN,NaN,NaN,NaN,BBG000BND699,NaN,NaN,NaN,...,fbn-uni-T02002,NXP Semiconductors,NaN,NaN,Equity,Equities,USD,True,NaN,USD
2,LUID_00003D72,US11135F1012,NaN,11135F101,NaN,NaN,BBG00KHY5S69,NaN,NaN,NaN,...,fbn-uni-T02002,Broadcom Inc.,NaN,NaN,Equity,Equities,USD,True,NaN,USD
3,LUID_00003D91,GB0008782301,878230,NaN,NaN,NaN,BBG000BF4KL1,NaN,NaN,NaN,...,fbn-uni-T02002,Taylor Wimpey,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
4,LUID_00003D9C,GB0000811801,81180,NaN,NaN,NaN,BBG000BD6DG6,NaN,NaN,NaN,...,fbn-uni-T02002,Barratt Developments,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,LUID_00003D74,US6792951054,NaN,679295105,NaN,NaN,BBG001YV1SM4,NaN,NaN,NaN,...,fbn-uni-T02002,"Okta, Inc.",NaN,NaN,Equity,Equities,USD,True,NaN,USD
266,LUID_00003DAZ,JE00BJVNSS43,BJVNSS4,NaN,NaN,NaN,BBG00NZJ0JG0,NaN,NaN,NaN,...,fbn-uni-T02002,Ferguson plc,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
267,LUID_00003D9L,GB0007980591,798059,NaN,NaN,NaN,BBG000C05BD1,NaN,NaN,NaN,...,fbn-uni-T02002,BP,NaN,NaN,Equity,Equities,GBP,True,NaN,GBP
268,LUID_00003D92,US0231351067,NaN,23135106,NaN,NaN,BBG000BB0NB9,NaN,NaN,NaN,...,fbn-uni-T02002,Amazon.com,NaN,NaN,Equity,Equities,USD,True,NaN,USD


## Using Luminesce variables
You can use variables to create and populate arbitrary tables of data that can then be used as part of a Luminesce query for either a data provider or a direct provider.

Declare a variable that represents a table of data using:
`@variable`

Or to represent a single value:
`@@variable`

Let's see variables in action. Here_instruments_in GBP is a table variable, set from the results of the query on equities.

In [155]:
%%luminesce
@instruments_in_GBP = select * from Lusid.Instrument.Equity where DomCcy = 'GBP';
Select Isin from @instruments_in_GBP;


,Isin
0,GB0008782301
1,GB0000811801
2,GB00B18V8630
3,GB0031743007
4,GB0000811801
...,...
149,GB00B1KJJ408
150,JE00B4T3BW64
151,GB0001367019
152,JE00BJVNSS43


In [158]:
%%luminesce
@sectors = use Drive.Csv
--file=/finbourne university/ftse_subset.csv
enduse;

@@selected_sector = SELECT 'Life Insurance';
SELECT * FROM @sectors WHERE Sector = @@selected_sector

,Name,Sector,Figi
0,Prudential plc,Life Insurance,BBG000BDY322
1,St. James's Place plc,Life Insurance,BBG000BRMPC4
2,Legal & General,Life Insurance,BBG000BDQCG6
3,Aviva,Life Insurance,BBG000DDCSJ1


## Introducing more advanced Luminesce extensions
You can use the CROSS APPLY or OUTER APPLY statement to iterate over a set of parameters. This is useful to execute a Luminesce query in parallel for each of the parameter inputs.
### cross-apply
Similar to INNER JOIN, in that only records that have matching values on both sides are returned. 
Here, we'll run some equity queries in parallel using Tools.split to create a set of currencies to filter by- the table will contain the currency joined to all of the returned equity tables:

In [1]:
%%luminesce
@currencies = select VALUE as val
from Tools.Split where Original = 'GBP USD CHF' AND Delimiters = ' ';

select c.val as selectedCurrency, results.*
from @currencies c
cross apply (
       SELECT equity.^ FROM Lusid.Instrument.Equity equity WHERE equity.DomCcy = c.val
       
) results;

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1
1,GBP,LUID_00003DHS,FBNUniversity,Diageo,Equity,Equities,GBP,True,1
2,GBP,LUID_00003D9Q,fbn-uni-T02002,Rentokil Initial,Equity,Equities,GBP,True,1
3,GBP,LUID_00003DGZ,FBNUniversity,Melrose Industries,Equity,Equities,GBP,True,1
4,GBP,LUID_00003DCP,FBNUniversity,Entain,Equity,Equities,GBP,True,1
...,...,...,...,...,...,...,...,...,...
467,USD,LUID_00003DEI,FBNUniversity,Starbucks Corporation,Equity,Equities,USD,True,2
468,USD,LUID_00003DGN,FBNUniversity,"Gilead Sciences, Inc.",Equity,Equities,USD,True,2
469,USD,LUID_00003DCV,FBNUniversity,"Micron Technology, Inc.",Equity,Equities,USD,True,2
470,USD,LUID_00003D5T,FBNUniversityModule-4-1,Microsoft,Equity,Equities,USD,True,2


### outer-apply
Similar to LEFT OUTER JOIN, in that all records on the left side are returned, even when there are no matching records on the right side.
We'll see that the CHF record is returned even though there are no instruments that have a DomCcy with value CHF.

In [2]:
%%luminesce
@currencies = select VALUE as val
from Tools.Split where Original = 'GBP USD CHF' AND Delimiters = ' ';

select c.val, results.*
from @currencies c
outer apply (
       SELECT equity.^ FROM Lusid.Instrument.Equity equity WHERE equity.DomCcy = c.val
       
) results;

,val,LusidInstrumentId,Scope,DisplayName,Type,InferredAssetClass,InferredDomCcy,IsActive,__row_key__
0,GBP,LUID_00003D4V,FBNUniversityModule-1,RELX,Equity,Equities,GBP,True,1.0
1,GBP,LUID_00003D4W,FBNUniversityModule-1,Halma,Equity,Equities,GBP,True,1.0
2,GBP,LUID_00003D4X,FBNUniversityModule-1,Unilever,Equity,Equities,GBP,True,1.0
3,GBP,LUID_00003D4Y,FBNUniversityModule-1,Just Eat Takeawy,Equity,Equities,GBP,True,1.0
4,GBP,LUID_00003D4Z,FBNUniversityModule-1,Mondi,Equity,Equities,GBP,True,1.0
...,...,...,...,...,...,...,...,...,...
468,USD,LUID_00003DHY,FBNUniversity,JD.com,Equity,Equities,USD,True,2.0
469,USD,LUID_00003DI0,FBNUniversity,Fox Corporation (Class A),Equity,Equities,USD,True,2.0
470,USD,LUID_00003DI1,FBNUniversity,Electronic Arts,Equity,Equities,USD,True,2.0
471,USD,LUID_00003DI5,FBNUniversity,Keurig Dr Pepper Inc.,Equity,Equities,USD,True,2.0
